# `Step 5: Autoencoder`
- Free
- Basic
- Denoising
- Dropout
- Combined 


# 0- `Config`

In [ ]:
import pandas as pd 
import numpy as np 
import pickle 

In [ ]:
try: 
    if manager == 1:
        print("s1_load.ipynb running from MANAGER")
except: 
    %run s0_config.ipynb 
    subject_autoencoder_0 = "Autoencoder_v4" # Embeddings 
    subject_autoencoder_1 = "Free_v0"

# 1- `Load`

In [ ]:
import_name = "data_raw"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_normalization}", f"{import_name}" + ".pkl")) , 'rb') as file:
    data_raw = pickle.load(file) 

import_name = "subject_data_full"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_normalization}", f"{import_name}" + ".pkl")) , 'rb') as file:
    subject_data_full = pickle.load(file) 


import_name = "subject_outlier_0"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_normalization}", f"{import_name}" + ".pkl")) , 'rb') as file:
    subject_outlier_0 = pickle.load(file) 


import_name = "subject_outlier_1"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_normalization}", f"{import_name}" + ".pkl")) , 'rb') as file:
    subject_outlier_1 = pickle.load(file) 
    
import_name = "subject_normalization_0"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_normalization}", f"{import_name}" + ".pkl")) , 'rb') as file:
    subject_normalization_0 = pickle.load(file) 


import_name = "subject_normalization_1"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_normalization}", f"{import_name}" + ".pkl")) , 'rb') as file:
    subject_normalization_1 = pickle.load(file) 


In [ ]:
print(f"Subject Outlier 0 (Embedding): {subject_outlier_0}")
print(f"Subject Outlier 1 (Expression): {subject_outlier_1}")
print(f"Subject normalization 0 (Embedding): {subject_normalization_0}")
print(f"Subject normalization 1 (Expression): {subject_normalization_1}")

---
# 2- `Preprocessing`

## 0-) `Overview`

In [ ]:
print(f"Subject Outlier 0 (Embedding): {subject_outlier_0}")
print(f"Subject Outlier 1 (Expression): {subject_outlier_1}")
print(f"Subject normalization 0 (Embedding): {subject_normalization_0}")
print(f"Subject normalization 1 (Expression): {subject_normalization_1}")
print(f"Subject Autoencoder 0 (Embedding): {subject_autoencoder_0}")
print(f"Subject Autoencoder 1 (Expression): {subject_autoencoder_1}")

## 1-) `Tool Setup`

---
# 3- `Bench`

### 1-) `Embeddings`

#### 0-) `Overview`

In [ ]:
subject_data_full.iloc[:3,:]

In [ ]:
import uuid 
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:

# Training and validation function
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    train_loss = 0
    for data in dataloader:
        inputs, _ = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(dataloader)

def validate_epoch(model, dataloader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            val_loss += loss.item()
    return val_loss / len(dataloader)

import torch.nn.functional as F

def train_epoch_vae(model, dataloader, optimizer):
    model.train()
    train_loss = 0
    for data in dataloader:
        inputs, _ = data
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(inputs)
        loss = vae_loss_function(recon_batch, inputs, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(dataloader)

def validate_epoch_vae(model, dataloader):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            recon_batch, mu, logvar = model(inputs)
            loss = vae_loss_function(recon_batch, inputs, mu, logvar)
            val_loss += loss.item()
    return val_loss / len(dataloader)

def vae_loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Hyperparameter options
options_batch_size = [16, 32, 64, 128]
options_batch_size = [32]
options_encoding_dim = [32, 64, 128, 256]
options_encoding_dim = [128]
options_loss_function = ["MSELoss", "L1Loss", "HuberLoss"]
options_loss_function = ["HuberLoss"]
options_optimizers = ["Adam", "AdamW", "SGD", "RMSprop"]
options_optimizers = ["RMSprop"]
options_learning_rates = [0.01, 0.001, 0.0001, 0.00001]
options_learning_rates = [0.001]
options_scheduler = ["None", "StepLR", "ReduceLROnPlateau", "ExponentialLR"]
options_scheduler = ["None"]
options_num_epochs = [10, 100, 200, 500]
options_num_epochs = [10] 
options_model_types = [ "Free_v0", "Autoencoder_v1", "Autoencoder_v2", "Autoencoder_v3", "Autoencoder_v4", "Autoencoder_v5", 
                       "BasicAutoencoder_v0", "DeepAutoencoder_v0", "SparseAutoencoder_v0", "ConvAutoencoder_v0", "VAE_v0"]

try:
    options_model_types = forced_options_model_types
except:
    options_model_types = [subject_autoencoder_0] # ["Autoencoder_v4"]

# Create an empty DataFrame to store results
total_stat_df = pd.DataFrame()

# Normalize the embeddings
embeddings = np.array(subject_data_full["Embeddings_Normalized"].tolist())
scaler = StandardScaler()
embeddings = scaler.fit_transform(embeddings)

# Split the data into training and validation sets
embeddings_train, embeddings_val = train_test_split(embeddings, test_size=0.2, random_state=42)

# Convert to torch tensors
embeddings_train_tensor = torch.tensor(embeddings_train, dtype=torch.float32)
embeddings_val_tensor = torch.tensor(embeddings_val, dtype=torch.float32)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(embeddings_train_tensor, embeddings_train_tensor)
val_dataset = TensorDataset(embeddings_val_tensor, embeddings_val_tensor)

# Initialize the model
input_dim = embeddings.shape[1]

---
# 3- `Bench`

In [ ]:
autoencoder_models = access_data_path(f"{process_barcode}/class/{folder_autoencoder}", "models" + ".ipynb") 
%run $autoencoder_models 

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_val_loss = float('inf')
        self.epochs_no_improve = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_val_loss - self.min_delta:
            self.best_val_loss = val_loss
            self.epochs_no_improve = 0
        else:
            self.epochs_no_improve += 1
            if self.epochs_no_improve >= self.patience:
                self.early_stop = True


In [ ]:
total_combinations = 0 
for model_type in options_model_types:
    for batch_size in options_batch_size:
        for encoding_dim in options_encoding_dim:
            for loss_function in options_loss_function:
                for optimizer_name in options_optimizers:
                    for learning_rate in options_learning_rates:
                        for scheduler_name in options_scheduler:
                            for num_epochs in options_num_epochs:
                                total_combinations += 1 
total_combinations

In [ ]:
# Create model dictionary and save models with information. 
model_dict = {} 
# Create unique id
current_combination = 0

# Early stopping parameters
early_stopping_patience = 10
early_stopping_min_delta = 0

In [ ]:

# Generate combinations of hyperparameters from the provided options
for model_type in options_model_types:
    for batch_size in options_batch_size:
        for encoding_dim in options_encoding_dim:
            for loss_function in options_loss_function:
                for optimizer_name in options_optimizers:
                    for learning_rate in options_learning_rates:
                        for scheduler_name in options_scheduler:
                            for num_epochs in options_num_epochs:

                                print("*" * 100)
                                print(f"Current Combination: {current_combination}")
                                current_combination += 1

                                print(f"Model Type: {model_type}")
                                print(f"Batch Size: {batch_size}")
                                print(f"Encoding Dim: {encoding_dim}")
                                print(f"Loss Function: {loss_function}")
                                print(f"Optimizer: {optimizer_name}")
                                print(f"Learning Rate: {learning_rate}")
                                print(f"Scheduler: {scheduler_name}")
                                print(f"Num Epochs: {num_epochs}")

                                # Initialize DataLoader
                                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                                print("*" * 100)
                                print("Selected Autoencoder: ", model_type)
                                print("*" * 100) 
                                if model_type == "Free_v0":
                                    # Directly use normalized embeddings without any autoencoder
                                    encoded_data = torch.tensor(embeddings, dtype=torch.float32)
                                else:
                                    # Initialize the model
                                    print("Current Model Type:", model_type)
                                    model = get_autoencoder_model(model_type = model_type, input_dim=input_dim, encoding_dim=encoding_dim)

                                    # Initialize loss function
                                    if loss_function == "MSELoss":
                                        criterion = nn.MSELoss()
                                    elif loss_function == "L1Loss":
                                        criterion = nn.L1Loss()
                                    elif loss_function == "HuberLoss":
                                        criterion = nn.SmoothL1Loss()

                                    # Optimizer initialization
                                    if optimizer_name == "Adam":
                                        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
                                    elif optimizer_name == "AdamW":
                                        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
                                    elif optimizer_name == "SGD":
                                        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
                                    elif optimizer_name == "RMSprop":
                                        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

                                    # Scheduler initialization
                                    if scheduler_name == "StepLR":
                                        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
                                    elif scheduler_name == "ReduceLROnPlateau":
                                        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
                                    elif scheduler_name == "ExponentialLR":
                                        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
                                    else:
                                        scheduler = None

                                    # Initialize early stopping
                                    early_stopping = EarlyStopping(patience=early_stopping_patience, min_delta=early_stopping_min_delta)

                                    # Training loop
                                    train_losses = []
                                    val_losses = []
                                    best_val_loss = float('inf')
                                    best_model_state = None

                                    try:
                                        for epoch in range(num_epochs):
                                            train_loss = train_epoch(model, train_loader, criterion, optimizer)
                                            val_loss = validate_epoch(model, val_loader, criterion)
                                            train_losses.append(train_loss)
                                            val_losses.append(val_loss)
                                            if scheduler is not None:
                                                if scheduler_name == "ReduceLROnPlateau":
                                                    scheduler.step(val_loss)
                                                else:
                                                    scheduler.step()
                                            print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
                                            
                                            # Save the best model
                                            if val_loss < best_val_loss:
                                                best_val_loss = val_loss
                                                best_model_state = model.state_dict()

                                            # Check early stopping condition
                                            early_stopping(val_loss)
                                            if early_stopping.early_stop:
                                                print("Early stopping triggered")
                                                break

                                        # Load the best model state before saving it to the model dictionary
                                        if best_model_state is not None:
                                            model.load_state_dict(best_model_state)

                                        # Collecting statistics
                                        model_id = "Selected_Autoencoder"
                                        stat_dict = {
                                            "model_id": model_id,
                                            "model_type": model_type,
                                            "train_first": train_losses[0],
                                            "train_last": train_losses[-1],
                                            "val_first": val_losses[0],
                                            "val_last": val_losses[-1],
                                            "batch_size": batch_size,
                                            "encoding_dim": encoding_dim,
                                            "loss_function": loss_function,
                                            "optimizer": optimizer_name,
                                            "learning_rate": learning_rate,
                                            "scheduler": scheduler_name,
                                            "num_epochs": num_epochs
                                        }
                                        model_dict[model_id] = model

                                        # Append results to the DataFrame
                                        df = pd.DataFrame(stat_dict, index=[0])
                                        total_stat_df = pd.concat([total_stat_df, df], axis=0)
                                        
                                        # Save the model
                                        torch.save(model.state_dict(), f"./model/autoencoder/selected.pth") 
                                        
                                        config_file = {"input_dim":input_dim, "encoding_dim":encoding_dim, "model_type":model_type}
                                        export_name = "config_file" 
                                        with open(os.path.join(access_data_path(f"{process_barcode}/model/autoencoder/", f"{export_name}" + ".pkl"))  , 'wb') as file: 
                                            pickle.dump(config_file, file)   
                                        
                                        
                                        # Encode the data and save it
                                        full_dataset = TensorDataset(torch.tensor(embeddings, dtype=torch.float32))
                                        encoded_data = []
                                        model.eval()
                                        with torch.no_grad():
                                            for data in DataLoader(full_dataset, batch_size=1):
                                                inputs = data[0]
                                                encoded_output = model.encoder(inputs)
                                                encoded_data.append(encoded_output)
                                        
                                        # Convert encoded data to a single tensor
                                        encoded_data = torch.cat(encoded_data)

                                        print(f"Encoded data shape: {encoded_data.shape}")  # Debug statement

                                        # Ensure the dimensions match
                                        assert encoded_data.shape[0] == len(subject_data_full), f"Row count mismatch: encoded_data has {encoded_data.shape[0]} rows, subject_data_full has {len(subject_data_full)} rows"

                                        # Save encoded data
                                        #torch.save(encoded_data, f"{model_id}_encoded_data.pth")

                                    except Exception as e:
                                        print(f"An error occurred: {e}")
                                        continue

In [ ]:
print(f"Current Shape {subject_data_full.shape}")
print(f"Current Shape {subject_data_full['Embeddings_Normalized'].iloc[0].shape}")

print(f"Tensor Shape {encoded_data.shape}") 

In [ ]:
# Convert the tensor to a list of lists
encoded_data_list = encoded_data.tolist()

# Ensure the dimensions match
assert len(encoded_data_list) == len(subject_data_full), "Row count mismatch between encoded_data and DataFrame"

# Add as a new column
subject_data_full['Embeddings_Autoencoder'] = encoded_data_list

subject_data_full 

---
---
# Visuals 

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns 
from umap import UMAP

In [ ]:
embeddings_list = subject_data_full["Embeddings_Normalized"].tolist()
raw_data_np = np.array(embeddings_list) 

In [ ]:
# Assuming encoded_data is derived from encoded_data
encoded_data_np = encoded_data.numpy()
# Extract labels from subject_data_full
cell_type_labels = subject_data_full["Cell_Type"].values[:len(encoded_data_np)] 

---
# 2D - UMAP 

## Raw 

In [ ]:
switch_umap = 1

In [ ]:
if switch_umap == 1: 
    # Apply 2D UMAP
    umap_2d_raw = UMAP(n_components=2, random_state=42)
    raw_umap_2d = umap_2d_raw.fit_transform(raw_data_np)
    # Ensure the cell type labels are the correct length
    cell_type_labels = subject_data_full["Cell_Type"].values
    # Verify the shape of the UMAP output
    print(f"Shape of raw_umap_2d: {raw_umap_2d.shape}") 


In [ ]:
if switch_umap == 1: 

    # Plot UMAP visualization
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=raw_umap_2d[:, 0], y=raw_umap_2d[:, 1], hue=cell_type_labels, palette='tab20', s=50, alpha=0.7)
    plt.title('UMAP Visualization of Raw Gene Marker Data')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.legend(title='Cell Types', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


## Encoded 

In [ ]:
if switch_umap == 1: 

    # Verify lengths
    print(f"Length of encoded_data_np: {encoded_data_np.shape[0]}")
    print(f"Length of cell_type_labels: {len(cell_type_labels)}") 
    # Apply 2D UMAP
    umap_2d_encoded = UMAP(n_components=2, random_state=42)
    encoded_umap_2d = umap_2d_encoded.fit_transform(encoded_data_np)


In [ ]:
if switch_umap == 1: 

    # Plot UMAP visualization
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=encoded_umap_2d[:, 0], y=encoded_umap_2d[:, 1], hue=cell_type_labels, palette='tab20', s=50, alpha=0.7)
    plt.title('UMAP Visualization of Encoded Gene Marker Data')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.legend(title='Cell Types', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


---
# 3D - UMAP 

In [ ]:
import plotly.express as px

## Raw  

In [ ]:
if switch_umap == 1: 

    # Apply 3D UMAP
    umap_3d_raw = UMAP(n_components=3, random_state=42)
    raw_umap_3d = umap_3d_raw.fit_transform(raw_data_np)
    # Ensure the cell type labels are the correct length
    cell_type_labels_raw = subject_data_full["Cell_Type"].values[:len(raw_umap_3d)]


In [ ]:
if switch_umap == 1: 

    # Create a DataFrame for Plotly
    df_raw = pd.DataFrame({
        'x': raw_umap_3d[:, 0],
        'y': raw_umap_3d[:, 1],
        'z': raw_umap_3d[:, 2],
        'label': cell_type_labels_raw
    })
    # Plot 3D UMAP visualization
    fig_raw = px.scatter_3d(df_raw, x='x', y='y', z='z', 
                            color='label', 
                            title='3D UMAP Visualization of Raw Gene Marker Data')

    # Update the marker size
    fig_raw.update_traces(marker=dict(size=3))  # Set the size to a smaller value

    # Show plot
    fig_raw.show()


## Encoded 

In [ ]:
if switch_umap == 1: 

    # Assuming encoded_data is derived from encoded_data
    encoded_data_np = encoded_data.numpy()
    # Ensure the cell type labels are the correct length
    cell_type_labels = subject_data_full["Cell_Type"].values[:len(encoded_data_np)]
    # Apply 3D UMAP
    umap_3d_encoded = UMAP(n_components=3, random_state=42)
    encoded_umap_3d = umap_3d_encoded.fit_transform(encoded_data_np)


In [ ]:
if switch_umap == 1: 

    # Create a DataFrame for Plotly
    df_encoded = pd.DataFrame({
        'x': encoded_umap_3d[:, 0],
        'y': encoded_umap_3d[:, 1],
        'z': encoded_umap_3d[:, 2],
        'label': cell_type_labels
    })
    # Plot 3D UMAP visualization
    fig_encoded = px.scatter_3d(df_encoded, x='x', y='y', z='z', 
                                color='label', 
                                title='3D UMAP Visualization of Encoded Gene Marker Data')

    # Update the marker size
    fig_encoded.update_traces(marker=dict(size=3))  # Set the size to a smaller value

    # Show plot
    fig_encoded.show()


---
# 2D - TSNE

### Raw 

In [ ]:
if switch_umap == 1: 

    # Convert the embeddings column to a 2D NumPy array
    embeddings_list = subject_data_full["Embeddings"].tolist()
    raw_data_np = np.array(embeddings_list)

    # Verify the shape of the array
    print(f"Shape of raw_data_np: {raw_data_np.shape}") 

In [ ]:
if switch_umap == 1: 

    # Apply 2D t-SNE
    tsne_2d_raw = TSNE(n_components=2, random_state=42)
    raw_tsne_2d = tsne_2d_raw.fit_transform(raw_data_np)

    # Ensure the cell type labels are the correct length
    cell_type_labels = subject_data_full["Cell_Type"].values

    # Verify the shape of the t-SNE output
    print(f"Shape of raw_tsne_2d: {raw_tsne_2d.shape}") 

In [ ]:
if switch_umap == 1: 

    # Plot t-SNE visualization
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=raw_tsne_2d[:, 0], y=raw_tsne_2d[:, 1], hue=cell_type_labels, palette='tab20', s=50, alpha=0.7)
    plt.title('t-SNE Visualization of Raw Gene Marker Data')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend(title='Cell Types', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


### Encoded

In [ ]:
if switch_umap == 1: 

    # Assuming encoded_data_np is derived from encoded_data
    encoded_data_np = encoded_data.numpy()

    # Extract labels from subject_data_full
    cell_type_labels = subject_data_full["Cell_Type"].values[:len(encoded_data_np)]

    # Verify lengths
    print(f"Length of encoded_data_np: {encoded_data_np.shape[0]}")
    print(f"Length of cell_type_labels: {len(cell_type_labels)}") 

In [ ]:
if switch_umap == 1: 

    # Apply 2D t-SNE
    tsne_2d_encoded = TSNE(n_components=2, random_state=42)
    encoded_tsne_2d = tsne_2d_encoded.fit_transform(encoded_data_np)

In [ ]:
if switch_umap == 1: 

    # Plot t-SNE visualization
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=encoded_tsne_2d[:, 0], y=encoded_tsne_2d[:, 1], hue=cell_type_labels, palette='tab20', s=50, alpha=0.7)
    plt.title('t-SNE Visualization of Encoded Gene Marker Data')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend(title='Cell Types', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


---
# 3D - TSNE 

In [ ]:
if switch_umap == 1: 

    # Assuming encoded_data_np is derived from encoded_data
    encoded_data_np = encoded_data.numpy()

    # Ensure the cell type labels are the correct length
    cell_type_labels = subject_data_full["Cell_Type"].values[:len(encoded_data_np)]

    # Create a mapping from cell type labels to numeric values
    unique_labels = np.unique(cell_type_labels)
    label_to_num = {label: idx for idx, label in enumerate(unique_labels)}
    numeric_labels = np.array([label_to_num[label] for label in cell_type_labels])

    # Verify the mapping
    print(f"Label to Numeric Mapping: {label_to_num}")
    print(f"Numeric Labels: {numeric_labels[:10]}")

### Raw 

In [ ]:
if switch_umap == 1: 

    # Apply 3D t-SNE
    tsne_3d_raw = TSNE(n_components=3, random_state=42)
    raw_tsne_3d = tsne_3d_raw.fit_transform(raw_data_np)

    # Ensure the cell type labels are the correct length
    cell_type_labels_raw = subject_data_full["Cell_Type"].values[:len(raw_tsne_3d)]

    # Verify lengths to ensure they match
    print(f"Length of raw_tsne_3d: {raw_tsne_3d.shape[0]}")
    print(f"Length of cell_type_labels_raw: {len(cell_type_labels_raw)}")

    # Create a DataFrame for Plotly
    df_raw = pd.DataFrame({
        'x': raw_tsne_3d[:, 0],
        'y': raw_tsne_3d[:, 1],
        'z': raw_tsne_3d[:, 2],
        'label': cell_type_labels_raw
    })

    # Plot 3D t-SNE visualization
    fig_raw = px.scatter_3d(df_raw, x='x', y='y', z='z', 
                            color='label', 
                            title='3D t-SNE Visualization of Raw Gene Marker Data')

    # Update the marker size
    fig_raw.update_traces(marker=dict(size=3))  # Set the size to a smaller value

    # Show plot
    fig_raw.show()


### Encoded

In [ ]:
if switch_umap == 1: 

    # Apply 3D t-SNE
    tsne_3d_encoded = TSNE(n_components=3, random_state=42)
    encoded_tsne_3d = tsne_3d_encoded.fit_transform(encoded_data_np)

    # Verify lengths to ensure they match
    print(f"Length of encoded_tsne_3d: {encoded_tsne_3d.shape[0]}")
    print(f"Length of cell_type_labels: {len(cell_type_labels)}")

    # Create a DataFrame for Plotly
    df_encoded = pd.DataFrame({
        'x': encoded_tsne_3d[:, 0],
        'y': encoded_tsne_3d[:, 1],
        'z': encoded_tsne_3d[:, 2],
        'label': cell_type_labels
    })

    # Plot 3D t-SNE visualization
    fig_encoded = px.scatter_3d(df_encoded, x='x', y='y', z='z', 
                                color='label', 
                                title='3D t-SNE Visualization of Encoded Gene Marker Data')

    # Update the marker size
    fig_encoded.update_traces(marker=dict(size=3))  # Set the size to a smaller value

    # Show plot
    fig_encoded.show()


---
---
# Rest 

In [ ]:
encoded_data 

In [ ]:
try: 
    subject_data_full["Expression_Autoencoder"] = subject_data_full["Expression"] 
except:
    pass 

---
# 4- `End`

In [ ]:
export_name = "data_raw" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(data_raw, file)   

export_name = "subject_data_full" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_data_full, file)   

    
export_name = "subject_outlier_0" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_outlier_0, file)   

export_name = "subject_outlier_1" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_outlier_1, file)   



export_name = "subject_normalization_0" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_normalization_0, file)   

export_name = "subject_normalization_1" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_normalization_1, file)   



export_name = "subject_autoencoder_0" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_autoencoder_0, file)   

export_name = "subject_autoencoder_1" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_autoencoder}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_autoencoder_1, file)   
    